https://www.geeksforgeeks.org/recommendation-system-in-python/

In [1]:
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
# Cargando el dataset de ratings
ratings = pd.read_csv("https://s3-us-west-2.amazonaws.com/recommender-tutorial/ratings.csv")
print(ratings.head())

   userId  movieId  rating  timestamp
0       1        1     4.0  964982703
1       1        3     4.0  964981247
2       1        6     4.0  964982224
3       1       47     5.0  964983815
4       1       50     5.0  964982931


In [3]:
# Cargando el dataset de movie
movies = pd.read_csv("https://s3-us-west-2.amazonaws.com/recommender-tutorial/movies.csv")
print(movies.head())

   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  


### Análisis Estadístico de Ratings

In [7]:
n_ratings = len(ratings)
n_movies = len(ratings['movieId'].unique())
n_users = len(ratings['userId'].unique())

print(f"Número de ratings: {n_ratings}")
print(f"Número de movieId's unicas: {n_movies}")
print(f"Número de usuarios unicos: {n_users}")
print(f"Promedio de ratings por usuario: {round(n_ratings/n_users, 2)}")
print(f"Promedio de ratings por película: {round(n_ratings/n_movies, 2)}")

Número de ratings: 100836
Número de movieId's unicas: 9724
Número de usuarios unicos: 610
Promedio de ratings por usuario: 165.3
Promedio de ratings por película: 10.37


### Frecuencia de ratings de los usuarios

In [10]:
user_freq = ratings[['userId', 'movieId']].groupby('userId').count().reset_index()
user_freq.columns = ['userId', 'n_ratings']
user_freq

,userId,n_ratings
0,1,232
1,2,29
2,3,39
3,4,216
4,5,44
...,...,...
605,606,1115
606,607,187
607,608,831
608,609,37


### Análisis del rating de las películas

In [13]:
# Encuentra las pelicultas calificadas más najo y más alto
mean_rating = ratings.groupby('movieId')[['rating']].mean()

# Películas peor calificadas
lowest_rated = mean_rating['rating'].idxmin()
movies.loc[movies['movieId'] == lowest_rated]

# Películas mejor calificadas
highest_rated = mean_rating['rating'].idxmax()
movies.loc[movies['movieId'] == highest_rated]

# Muestra el número de personas que calificaron las mejor calificadas
ratings[ratings['movieId'] == highest_rated]

# Muestra el número de personas que calificaron las peor calificadas
ratings[ratings['movieId'] == lowest_rated]

# Las anteriores peliculas tienen datos muy bajos. Vamos a usar el promedio bayesiano
movie_stats = ratings.groupby('movieId')[['rating']].agg(['count', 'mean'])
movie_stats.columns = movie_stats.columns.droplevel()

### Creando la matriz user-item

In [14]:
# Ahora creamos la matriz user-item scipy csr matrix
from scipy.sparse import csr_matrix

def create_matrix(df):
    
    N = len(df['userId'].unique())
    M = len(df['movieId'].unique())
    
    # Mapea los Id's a los indices
    user_mapper = dict(zip(np.unique(df['userId']), list(range(N))))
    movie_mapper = dict(zip(np.unique(df['movieId']), list(range(M))))
    
    # Mapea indices a los Id's
    user_inv_mapper = dict(zip(list(range(N)), np.unique(df['userId'])))
    movie_inv_mapper = dict(zip(list(range(M)), np.unique(df['movieId'])))
    
    user_index = [user_mapper[i] for i in df['userId']]
    movie_index = [movie_mapper[i] for i in df['movieId']]
    
    X = csr_matrix((df['rating'], (movie_index, user_index)), shape = (M, N))
    
    return X, user_mapper, movie_mapper, user_inv_mapper, movie_inv_mapper
     
X, user_mapper, movie_mapper, user_inv_mapper, movie_inv_mapper = create_matrix(ratings)

### Análisis de películas similares

In [20]:
# Encuentra películas similares usando K-Nearest Neighbors (KNN)
from sklearn.neighbors import NearestNeighbors

def find_similar_movies(movie_id, X, k, metric='cosine', show_distance=False):
    
    neighbour_ids = []
    
    movie_ind = movie_mapper[movie_id]
    movi_vec = X[movie_ind]
    k += 1
    kNN = NearestNeighbors(n_neighbors=k, algorithm="brute", metric=metric)
    kNN.fit(X)
    movi_vec = movi_vec.reshape(1, -1)
    neighbour = kNN.kneighbors(movi_vec, return_distance=show_distance)
    
    for i in range(0, k):
        n = neighbour.item(i)
        neighbour_ids.append(movie_inv_mapper[n])
    neighbour_ids.pop(0)
    
    return neighbour_ids

movie_titles = dict(zip(movies['movieId'], movies['title']))

movie_id = 3

similar_ids = find_similar_movies(movie_id, X, k=10)
movie_title = movie_titles[movie_id]

print(f"Desde que has visto: {movie_title}")
for i in similar_ids:
    print(movie_titles[i])

Desde que has visto: Grumpier Old Men (1995)
Grumpy Old Men (1993)
Striptease (1996)
Nutty Professor, The (1996)
Twister (1996)
Father of the Bride Part II (1995)
Broken Arrow (1996)
Bio-Dome (1996)
Truth About Cats & Dogs, The (1996)
Sabrina (1995)
Birdcage, The (1996)


### Recomendaciones de películas con refenrencias en las preferencias del usuario

In [26]:
def recommend_movies_for_user(user_id, X, user_mapper, movie_mapper, movie_inv_mapper, k=10):
    df1 = ratings[ratings['userId'] == user_id]
    
    if df1.empty:
        print(f"El usuario con ID {user_id} no existe")
        return
    
    movie_id = df1[df1['rating'] == max(df1['rating'])]['movieId'].iloc[0]
    
    movie_titles = dict(zip(movies['movieId'], movies['title']))
    
    similar_ids = find_similar_movies(movie_id, X, k)
    movie_title = movie_titles.get(movie_id, "Película no encontrada")
    
    if movie_title == "Película no encontrada":
        print(f"La película con ID {movie_id} no ha sido encontrada.")
        
    print(f"Si has visto la película {movie_id} también te podría interesar: ")
    for i in similar_ids:
        print(movie_titles.get(i, "Película no encontrada"))

In [31]:
user_id = 150
recommend_movies_for_user(user_id, X, user_mapper, movie_mapper, movie_inv_mapper)

Si has visto la película 32 también te podría interesar: 
Pulp Fiction (1994)
Terminator 2: Judgment Day (1991)
Independence Day (a.k.a. ID4) (1996)
Seven (a.k.a. Se7en) (1995)
Fargo (1996)
Fugitive, The (1993)
Usual Suspects, The (1995)
Jurassic Park (1993)
Star Wars: Episode IV - A New Hope (1977)
Heat (1995)


In [33]:
user_id = 2300
recommend_movies_for_user(user_id, X, user_mapper, movie_mapper, movie_inv_mapper, k=10)

El usuario con ID 2300 no existe
